In [4]:
pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 3.1 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd
import os
import folium 

In [11]:
cwd=os.getcwd()
accidents_2022idf_path=os.path.join(cwd,"accidents_2022_idf.xlsx")

accidents_2022idf_carac=pd.read_excel(accidents_2022idf_path)

#reshape des data
accidents_2022idf_carac["Latitude"]=accidents_2022idf_carac["Latitude"].str.replace(',','.').astype(float)
accidents_2022idf_carac["Longitude"]=accidents_2022idf_carac["Longitude"].str.replace(',','.').astype(float)

In [12]:
accidents_2022idf_carac.columns

Index(['Dépt', 'Commune', 'Date', 'Tué(s)', 'Blessés', 'dont hospitalisés',
       'dont légers', 'Milieu', 'Autoroute', 'Adresse', 'Latitude',
       'Longitude', 'Résumé'],
      dtype='object')

In [13]:
#on trie les données d'accident en fonction de leur localisation
dict_accidents_2022idf = accidents_2022idf_carac.groupby(["Commune","Adresse"]).groups
#on trie le dictionnaire obtenu
dict_trie_accidents_2022idf=dict(sorted(dict_accidents_2022idf.items(),key=lambda item : len(item[1]),reverse=True))


carte=folium.Map([48.866667,2.333333],zoom_start=12)
#on affiche les 25 localisations avec le plus d'accidents en Idf
i=0
for key in dict_trie_accidents_2022idf.keys():
    list1=[]
    for elem in dict_trie_accidents_2022idf[key]:
        list1.append(accidents_2022idf_carac.iloc[elem]["Latitude"])
        m1,m2=list1.index(min(list1)),list1.index(max(list1))
        #on affiche une ligne entre les points les plus éloignés
    coord_ligne1=[accidents_2022idf_carac.iloc[dict_trie_accidents_2022idf[key][m1]]['Longitude'],accidents_2022idf_carac.iloc[dict_trie_accidents_2022idf[key][m1]]['Latitude']]
    coord_ligne2=[accidents_2022idf_carac.iloc[dict_trie_accidents_2022idf[key][m2]]['Longitude'],accidents_2022idf_carac.iloc[dict_trie_accidents_2022idf[key][m2]]['Latitude']]
    folium.PolyLine([coord_ligne1,coord_ligne2],popup=key,weight = 0.7 * len(dict_trie_accidents_2022idf[key]/len(dict_trie_accidents_2022idf[('93066 - Saint-Denis','AUTOROUTE A1')]))).add_to(carte)
    i=i+1
    if i > 25:
        break
    
carte.save("accidents_idf_2022.html")

        

In [9]:
carte